<a href="https://colab.research.google.com/github/kristina-skoptsova/diplom/blob/main/verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Файл для проверки метрик моделей.**

*   Все файлы находятся в открытом репозитории: https://github.com/kristina-skoptsova/diplom
*   Загрузите файл *requirements.txt*
*   Загрузите файл *dataset_muiv.csv*
*   Загрузите все файлы из папки model
*   Запустите по порядку блоки кода







In [3]:
# Установка зависимостей
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00


Импорт необходимых бибилиотек

In [40]:
import pandas as pd
import numpy as np
import joblib
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, r2_score
from keras.models import load_model

Определение функций создание последовательности и оценки

In [49]:
def create_sequences(data, window_size):
    inputs, outputs, groups = [], [], []
    for direction in data['Направление'].unique():
        dir_data = data[data['Направление'] == direction].sort_values('Год поступления')
        for i in range(len(dir_data) - window_size):
            seq = dir_data.iloc[i:i+window_size].drop(['Направление', 'Год поступления', 'Количество поступивших'], axis=1).values
            target = dir_data.iloc[i+window_size]['Количество поступивших']
            direction_label = dir_data.iloc[i+window_size]['Направление']  # Направление для следующего шага
            inputs.append(seq)
            outputs.append(target)
            groups.append(direction_label)
    return np.array(inputs), np.array(outputs), np.array(groups)

def model_evaluation(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MAE: {mae:.4f}")
    print(f"R²: {r2:.4f}")
    print("-" * 30)


Подготовка данных

In [46]:
# Загрузка данных
df = pd.read_csv('/content/dataset_muiv.csv')
df = df.drop(columns=["Уровень безработицы", "Регион рождения", "Доля наличия договора"])

# Нормализация (кроме целевой переменной для ансамблей)
exclude_columns = ['Направление', 'Год поступления', 'Количество поступивших']
scaler = RobustScaler()
df[df.columns.difference(exclude_columns)] = scaler.fit_transform(df[df.columns.difference(exclude_columns)])

# Разделение X и y
X = df.drop(columns=['Количество поступивших'])
y = df['Количество поступивших'].values

Проверка ансамблевых моделей

In [47]:
# Разделение данных (для ансамблей)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверка RandomForest
model_rf = joblib.load('/content/model1_rf.joblib')
model_evaluation(model_rf, X_test, y_test)

# Проверка GradientBoosting
model_gbr = joblib.load('/content/model2_gbr.joblib')
model_evaluation(model_gbr, X_test, y_test)

# Проверка XGBoost
model_xgb = xgboost.XGBRegressor()
model_xgb.load_model('/content/model3_xgb.json')
model_evaluation(model_xgb, X_test, y_test)

MAE: 3.2186, R²: 0.8665
------------------------------
MAE: 3.4307, R²: 0.8540
------------------------------
MAE: 2.2972, R²: 0.9280
------------------------------


Проверка моделей LSTM и GRU

In [54]:
# Для LSTM/GRU
X, y, direction_labels = create_sequences(df, window_size=3)

# Нормализация целевой переменной
y_scaler = RobustScaler()
y = y_scaler.fit_transform(y.reshape(-1, 1)).flatten()

# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test, direction_train, direction_test = train_test_split(
    X, y, direction_labels, test_size=0.2, random_state=42)

# Проверка LSTM
model_lstm1 = load_model('/content/model4_lstm.keras')
loss, mae = model_lstm1.evaluate(X_test, y_test, verbose=0)
print(f"LSTM1 — Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Проверка GRU
model_gru = load_model('/content/model5_gru.keras')
loss, mae = model_gru.evaluate(X_test, y_test, verbose=0)
print(f"GRU — Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Проверка LSTM после оптимизации
X, y, direction_labels = create_sequences(df, window_size=5)
y_scaler = RobustScaler()
y = y_scaler.fit_transform(y.reshape(-1, 1)).flatten()
X_train, X_test, y_train, y_test, direction_train, direction_test = train_test_split(
    X, y, direction_labels, test_size=0.2, random_state=42)
model_lstm2 = load_model('/content/model7_lstm.keras')
loss, mae = model_lstm2.evaluate(X_test, y_test, verbose=0)
print(f"LSTM2 — Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

LSTM1 — Test Loss: 0.3116, Test MAE: 0.3116
GRU — Test Loss: 0.3167, Test MAE: 0.3167
LSTM2 — Test Loss: 0.2240, Test MAE: 0.2240


Проверка модели MLP

In [56]:
# Проверка MLP
X = df.drop(columns=['Количество поступивших'])
y = df['Количество поступивших']
direction = df['Направление'].values
y = df['Количество поступивших'].values
# Веса для классов
direction_counts = pd.Series(df['Направление']).value_counts()
direction_weights = (1 / direction_counts).to_dict()
sample_weights = np.array([direction_weights[d] for d in df['Направление']])

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, sample_weights, test_size=0.2, random_state=42)

model_mlp = load_model('/content/model6_mlp.keras')

loss, mae = model_mlp.evaluate(X_test, y_test, sample_weight=weights_test, verbose=0)
print(f"MLP — Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

MLP — Test Loss: 0.2802, Test MAE: 32.7780
